<a href="https://colab.research.google.com/github/PolChip/2course_mash_lear/blob/main/1_2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

18. Анализ и прогнозирование дорожных происшествий по историческим данным.

In [2]:
%pip install kaggle

In [4]:
import ipywidgets as widgets
import os

def get_and_store_secret(placeholder, output_secret_path):
  password_field = widgets.Password(
      description=f"Enter Key:",
      placeholder=placeholder,
      disabled=False
  )

  output = widgets.Output()

  def on_button_click(b):
    with output:
      token = password_field.value
      with open(os.path.expanduser(output_secret_path), "w") as f:
        f.write(token)
      password_field.close()
      button.close()

  button = widgets.Button(description="Save Key")
  button.on_click(on_button_click)

  display(password_field, button, output)

In [5]:
kaggle_user_path = os.path.expanduser("~/.cache/kaggle-user.txt")
get_and_store_secret("KAGGLE_USERNAME", kaggle_user_path)

Password(description='Enter Key:', placeholder='KAGGLE_USERNAME')

Button(description='Save Key', style=ButtonStyle())

Output()

In [9]:
kaggle_key_path = os.path.expanduser("~/.cache/kaggle.token")
get_and_store_secret("KAGGLE_KEY", kaggle_key_path)

Password(description='Enter Key:', placeholder='KAGGLE_KEY')

Button(description='Save Key', style=ButtonStyle())

Output()

In [11]:
with open(kaggle_user_path) as f:
    os.environ["KAGGLE_USERNAME"] = f.read().strip()

with open(kaggle_key_path) as f:
    os.environ["KAGGLE_KEY"] = f.read().strip()

In [18]:
import pandas as pd          # Основная библиотека для работы с табличными данными
from io import StringIO      # Для работы со строками как с файлами
import os                    # Для работы с операционной системой (файлы, пути)

# Получаем список файлов
dataset_files = !{"kaggle datasets files himelsarder/road-accident-survival-dataset --csv"}
dataset_files = "\n".join(dataset_files) # Объединяем список строк в одну строку (с переносами между ними)
files_df = pd.read_csv(StringIO(dataset_files)) # StringIO(dataset_files) создает файлоподобный объект из строки

filename = files_df.iloc[0]['name'] # files_df.iloc[0] берет первую строку DataFrame, получает значение из колонки 'name'
print(f"Скачиваем файл: {filename}")

# Скачиваем файл ((-f + filename) указывает какой именно файл скачать)
!{"kaggle datasets download himelsarder/road-accident-survival-dataset -f " + filename}

# Проверяем скачался ли файл
downloaded_files = os.listdir('.') # Возвращаем список всех файлов и папок в текущей директории('.' означает "текущая директория")
csv_files = [f for f in downloaded_files if f.endswith('.csv')]

df = pd.read_csv(csv_files[0]) # Загрузка данных в DataFrame
print(f"Успешно загружен: {csv_files[0]}")
print(f"Размер данных: {df.shape}") # (строки, колонки)
print("\nДанных:")
df.head()

Скачиваем файл: accident.csv
Dataset URL: https://www.kaggle.com/datasets/himelsarder/road-accident-survival-dataset
License(s): CC0-1.0
  0% 0.00/4.44k [00:00<?, ?B/s]
100% 4.44k/4.44k [00:00<00:00, 14.9MB/s]
Успешно загружен: accident.csv
Размер данных: (200, 6)

Данных:


,Age,Gender,Speed_of_Impact,Helmet_Used,Seatbelt_Used,Survived
0,56,Female,27.0,No,No,1
1,69,Female,46.0,No,Yes,1
2,46,Male,46.0,Yes,Yes,0
3,32,Male,117.0,No,Yes,0
4,60,Female,40.0,Yes,Yes,0


In [19]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              200 non-null    int64  
 1   Gender           199 non-null    object 
 2   Speed_of_Impact  197 non-null    float64
 3   Helmet_Used      200 non-null    object 
 4   Seatbelt_Used    200 non-null    object 
 5   Survived         200 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 9.5+ KB


,Age,Speed_of_Impact,Survived
count,200.00000,197.000000,200.00000
mean,43.42500,70.441624,0.50500
std,14.94191,30.125298,0.50123
min,18.00000,20.000000,0.00000
25%,31.00000,43.000000,0.00000
50%,43.50000,71.000000,1.00000
75%,56.00000,95.000000,1.00000
max,69.00000,119.000000,1.00000


Чтобы предсказывать погибнет человек или нет, мы будем использовать модель бинарной классификации